In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
from albert import modeling
from albert import optimization
from albert import tokenization
import tensorflow as tf
import numpy as np

In [3]:
tokenizer = tokenization.FullTokenizer(
      vocab_file='albert-base-2020-04-10/sp10m.cased.v10.vocab', do_lower_case=False,
      spm_model_file='albert-base-2020-04-10/sp10m.cased.v10.model')


INFO:tensorflow:loading sentence piece model


In [4]:
albert_config = modeling.AlbertConfig.from_json_file('albert-base-2020-04-10/config.json')
albert_config

In [8]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/corpus/subjectivity/subjectivity-negative-bm.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/corpus/subjectivity/subjectivity-positive-bm.txt

In [9]:
with open('subjectivity-negative-bm.txt','r') as fopen:
    texts = fopen.read().split('\n')
labels = [0] * len(texts)

with open('subjectivity-positive-bm.txt','r') as fopen:
    positive_texts = fopen.read().split('\n')
labels += [1] * len(positive_texts)
texts += positive_texts

assert len(labels) == len(texts)

In [10]:
from tqdm import tqdm

input_ids, input_masks = [], []

for text in tqdm(texts):
    tokens_a = tokenizer.tokenize(text)
    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_id)
    
    input_ids.append(input_id)
    input_masks.append(input_mask)

100%|██████████| 9962/9962 [00:01<00:00, 7518.92it/s]


In [11]:
epoch = 3
batch_size = 60
warmup_proportion = 0.1
num_train_steps = int(len(texts) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [12]:
def create_initializer(initializer_range=0.02):
    return tf.truncated_normal_initializer(stddev=initializer_range)

class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
        training = True,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.MASK = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        
        model = modeling.AlbertModel(
            config=albert_config,
            is_training=training,
            input_ids=self.X,
            input_mask=self.MASK,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_sequence_output()
        output_layer = tf.layers.dense(
            output_layer,
            albert_config.hidden_size,
            activation=tf.tanh,
            kernel_initializer=create_initializer())
        self.logits_seq = tf.layers.dense(output_layer, dimension_output,
                                         kernel_initializer=create_initializer())
        self.logits_seq = tf.identity(self.logits_seq, name = 'logits_seq')
        self.logits = self.logits_seq[:, 0]
        self.logits = tf.identity(self.logits, name = 'logits')
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        
        self.optimizer = optimization.create_optimizer(self.cost, learning_rate, 
                                                       num_train_steps, num_warmup_steps, False)
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [13]:
BERT_INIT_CHKPNT = 'albert-base-2020-04-10/model.ckpt-400000'

In [14]:
dimension_output = 2
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, BERT_INIT_CHKPNT)





Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


INFO:tensorflow:++++++ warmup starts at step 0, for 49 steps ++++++
INFO:tensorflow:using adamw

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Restoring parameters from albert-base-2020-04-10/model.ckpt-400000


In [15]:
from sklearn.model_selection import train_test_split

train_input_ids, test_input_ids, train_Y, test_Y, train_mask, test_mask = train_test_split(
    input_ids, labels, input_masks, test_size = 0.2
)

In [16]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [17]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 1, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_input_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_input_ids))
        batch_x = train_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_mask = train_mask[i: index]
        batch_mask = pad_sequences(batch_mask, padding='post')
        batch_y = train_Y[i: index]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.MASK: batch_mask
            },
        )
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_input_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_input_ids))
        batch_x = test_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_y = test_Y[i: index]
        batch_mask = test_mask[i: index]
        batch_mask = pad_sequences(batch_mask, padding='post')
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.MASK: batch_mask
            },
        )
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)
        
    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 0/133 [00:00<?, ?it/s]

epoch: 0, pass acc: 0.000000, current acc: 0.907353
time taken: 83.79385900497437
epoch: 0, training loss: 0.380865, training acc: 0.820897, valid loss: 0.232055, valid acc: 0.907353



test minibatch loop: 100%|██████████| 34/34 [00:06<00:00,  5.03it/s, accuracy=1, cost=0.0461]   

time taken: 78.35151791572571
epoch: 1, training loss: 0.172869, training acc: 0.934947, valid loss: 0.234636, valid acc: 0.905392

break epoch:2



In [18]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'albert-base-subjectivity/model.ckpt')

'albert-base-subjectivity/model.ckpt'

In [19]:
dimension_output = 2
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate,
    training = False
)

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.trainable_variables())
saver.restore(sess, 'albert-base-subjectivity/model.ckpt')

/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


INFO:tensorflow:++++++ warmup starts at step 0, for 49 steps ++++++
INFO:tensorflow:using adamw
INFO:tensorflow:Restoring parameters from albert-base-subjectivity/model.ckpt


In [20]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'Placeholder_2',
 'bert/embeddings/word_embeddings',
 'bert/embeddings/token_type_embeddings',
 'bert/embeddings/position_embeddings',
 'bert/embeddings/LayerNorm/gamma',
 'bert/encoder/embedding_hidden_mapping_in/kernel',
 'bert/encoder/embedding_hidden_mapping_in/bias',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/query/kernel',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/query/bias',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/key/kernel',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/key/bias',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/value/kernel',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/value/bias',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/output/dense/kernel',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/output/dense/bias',
 'bert/encoder/transformer/group_0/inner_group_0/Lay

In [21]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_input_ids), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    index = min(i + batch_size, len(test_input_ids))
    batch_x = test_input_ids[i: index]
    batch_x = pad_sequences(batch_x, padding='post')
    batch_mask = test_mask[i: index]
    batch_mask = pad_sequences(batch_mask, padding='post')
    batch_y = test_Y[i: index]
    
    predict_Y += np.argmax(sess.run(model.logits,
            feed_dict = {
            model.X: batch_x,
            model.MASK: batch_mask
            },
    ), 1, ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 34/34 [00:03<00:00, 10.50it/s]


In [22]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, predict_Y, target_names = ['negative', 'positive'],digits = 5
    )
)

              precision    recall  f1-score   support

    negative    0.87616   0.94006   0.90699      1001
    positive    0.93471   0.86593   0.89901       992

    accuracy                        0.90316      1993
   macro avg    0.90544   0.90299   0.90300      1993
weighted avg    0.90531   0.90316   0.90301      1993



In [23]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [24]:
freeze_graph('albert-base-subjectivity', strings)

INFO:tensorflow:Restoring parameters from albert-base-subjectivity/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 29 variables.
INFO:tensorflow:Converted 29 variables to const ops.
3242 ops in the final graph.


In [25]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [26]:
import boto3

bucketName = 'huseinhouse-storage'
Key = 'albert-base-subjectivity/frozen_model.pb'
outPutname = "v34/subjective/albert-base-subjective.pb"

s3 = boto3.client('s3')

s3.upload_file(Key,bucketName,outPutname)